<a href="https://colab.research.google.com/github/wangzhenyagit/myColab/blob/main/langgraph_code_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph bs4

## Docs

Load [LangChain Expression Language](https://python.langchain.com/docs/expression_language/) (LCEL) docs as an example.

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Langchain docs
url = "https://python.langchain.com/docs/get_started/quickstart/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [9]:
import os
os.environ['OPENAI_API_KEY'] = 'your_api_key_here'

In [5]:
concatenated_content

'\n\n\n\n\nQuickstart | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystem🦜🛠️ LangSmith🦜🕸️LangGraph🦜️🏓 LangServeSecurityGet startedQuickstartOn this pageQuickstartIn this quickstart we\'ll show you how to:Get setup with LangChain, LangSmith and LangServeUse the most basic and common components of LangChain: prompt templates, models, and output parsersUse LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chainingBuild a simple application with LangChainT

In [33]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-yuCEDh0twTz70cRVaItoT3BlbkFJ1MWMxw288GpXUAr1vRK3'

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [("system","""You are a coding assistant with expertise in LangChain. \n
    Here is a full set of LangChain documentation:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided documentation. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    If code use ChatOpenAI class, do not use the api_key parameter. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:"""),
    ("placeholder", "{messages}")]
)

# Data model
class code(BaseModel):
    """Code output"""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")
    description = "Schema for code solutions to questions about LCEL."

expt_llm = "gpt-4-1106-preview"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain = code_gen_prompt | llm.with_structured_output(code)

In [20]:
question = "How do I ask simple question use langchain?"
solution = code_gen_chain.invoke({"context":concatenated_content,"messages":[("user",question)]})

In [21]:
solution.code

'# Initialize the OpenAI model\nllm = ChatOpenAI(api_key=\'your_openai_api_key\')\n\n# Define a prompt template\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "You are a world class technical documentation writer."),\n    ("user", "{input}")\n])\n\n# Combine the prompt with the model\nchain = prompt | llm\n\n# Add an output parser to convert the chat message to a string\noutput_parser = StrOutputParser()\nchain = chain | output_parser\n\n# Invoke the chain with a simple question\nresponse = chain.invoke({"input": "How can LangSmith help with testing?"})\nprint(response)'

## State

Our state is a dict that will contain keys (errors, question, code generation) relevant to code generation.

In [29]:
from typing import Dict, TypedDict, List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error : str
    messages : List
    generation : str
    iterations : int

## Graph

Our graph lays out the logical flow shown in the figure above.

In [34]:
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough

### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = 'do not reflect'

### Nodes

def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [("user","Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:")]

    # Solution
    code_solution = code_gen_chain.invoke({"context": concatenated_content, "messages" : messages})
    messages += [("assistant",f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}")]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}

def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    prefix = code_solution.prefix
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "yes"}

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "yes"}

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "no"}

def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection
    reflection_message = [("user", """You tried to solve this problem and failed a unit test. Reflect on this failure
                                    given the provided documentation. Write a few key suggestions based on the
                                    documentation to avoid making this mistake again.""")]

    # Add reflection
    reflections = code_gen_chain.invoke({"context"  : concatenated_content, "messages" : messages})
    messages += [("assistant" , f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}

### Edges

def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == 'reflect':
            return "reflect"
        else:
            return "generate"

In [35]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [36]:
question = "How can I use langchain ask question: what is large language model ?"
app.invoke({"messages":[("user",question)],"iterations":0})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
A large language model is a type of artificial intelligence model that has been trained on massive amounts of text data to understand and generate human-like language. These models are typically based on deep learning techniques, such as neural networks, and have the capability to process and generate text at a large scale. Examples of large language models include GPT-3 (Generative Pre-trained Transformer 3) developed by OpenAI and BERT (Bidirectional Encoder Representations from Transformers) developed by Google.
---NO CODE TEST FAILURES---
---DECISION: FINISH---


{'error': 'no',
 'messages': [('user',
   'How can I use langchain ask question: what is large language model ?'),
  ('assistant',
   'Using LangChain to ask a question about large language models involves setting up the necessary components such as the LLM (Large Language Model) integration, prompt templates, and output parsers. The following code demonstrates how to initialize a ChatOpenAI instance, create a prompt template, and invoke the LLM to ask the question \'What is a large language model?\'. The code assumes that the necessary packages have been installed and the OPENAI_API_KEY environment variable has been set. \n Imports: from langchain_openai import ChatOpenAI\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.output_parsers import StrOutputParser \n Code: # Initialize the ChatOpenAI instance\nllm = ChatOpenAI()\n\n# Create a prompt template\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "You are an AI knowledgeable about language mod